In [2]:
import boto3
import argparse
import json
import logging
import pathlib
from rich import print
# ###################################################
# # Input Variables
# parser = argparse.ArgumentParser()
# optionalNamed = parser.add_argument_group('optional named arguments')
# optionalNamed.add_argument("-p","--profile",default='default',type=str,help="The AWS profile to use")
# optionalNamed.add_argument("--env",type=str,help="Use Environment Variables rather than an AWS profile")
# optionalNamed.add_argument("--region",type=str,help="To enumerate the policies for a specific region (defaults to all regions)")
# optionalNamed.add_argument("--log",default='info',type=str,help="The logging level (debug,info,warning,error,critical)")

# # Get the passed arguements
# args = parser.parse_args()

class argss:
    profile = '981710073011_SSOPentestRole'
    # profile = '002627273584_SSOPentestRole'

args = argss()
# args["profile"] = "981710073011_SSOPentestRole"
s = boto3.session.Session(profile_name=args.profile,region_name='af-south-1')
ec2 = s.client("ec2",region_name='af-south-1')


########################### ENIS
class ENI(dict):
    # attributes_to_load = "Attachment","SubnetId","Groups"

    def __init__(self, eni_dict: dict) -> None:
        for k,v in eni_dict.items():
            # if k in ENI.attributes_to_load:
                self[k] = v
 
class ENIS(dict):
    def __init__(self,list_of_eni_dicts) -> None:
        for eni_dict in list_of_eni_dicts:
            network_id = eni_dict["NetworkInterfaceId"]
            self[network_id] = ENI(eni_dict)

    def attach_resource(self, eni_id: str, resource_type: str,resource_data: dict):
        if "attached_resource_types" not in self[eni_id].keys():
            self[eni_id]["attached_resource_types"] = [resource_type]
        elif resource_type not in self[eni_id]["attached_resource_types"]:
            self[eni_id]["attached_resource_types"].append(resource_type)
        # Can be attached to multiple eni's. Instanctiate the list if it's the first entry, else append to it
        if "attached_resources" not in self[eni_id].keys():
            self[eni_id]["attached_resources"] = [resource_data]
        elif resource_data not in self[eni_id]["attached_resources"]:
            self[eni_id]["attached_resources"].append(resource_data)

eni_data = ec2.describe_network_interfaces()["NetworkInterfaces"]
enis = ENIS(eni_data)

########################### Subnets
class SUBNET(dict):
    # attributes_to_load = "Attachment","SubnetId","Groups"
    def __init__(self, subnet_dict: dict) -> None:
        for k,v in subnet_dict.items():
            # if k in ENI.attributes_to_load:
            self[k] = v
        self.find_enis_in_subnet()
        # print(f"Found {len(self.enis_in_sunet)} ENIs in {self['SubnetId']}")

    def find_enis_in_subnet(self):
        self.enis_in_sunet = [([x['NetworkInterfaceId'],x['PrivateIpAddress']]) 
                    for k,x in enis.items() 
                    if x["SubnetId"] == self["SubnetId"]]
subnet_data = ec2.describe_subnets()["Subnets"]
subnets = [SUBNET(x) for x in subnet_data]
                    
print(f"Total subnets: {len(subnets)}")

# for subnet in subnets:
#     # print(subnet['SubnetId'])
#     enis_in_subnet = [', '.join([x['NetworkInterfaceId'],x['PrivateIpAddress']]) 
#                     for k,x in enis.items() 
#                     if x["SubnetId"] == subnet["SubnetId"]]
    # print(f"ENIs in subnet {subnet['SubnetId']} ({subnet['CidrBlock']})")
    # print(enis_in_subnet)

########################### VPC
class VPC(dict):
    def __init__(self, vpc_dict: dict) -> None:
        for k,v in vpc_dict.items():
            # if k in ENI.attributes_to_load:
                self[k] = v
        self.find_subnets()
        # print(f"Found {len(self.subnets)} subnets in {self['VpcId']}")
        # print(self.subnets)
        self["AssociatedCidrBlocks"] = [x['CidrBlock'] for x in self['CidrBlockAssociationSet'] if x['CidrBlockState']['State'] == 'associated']

    def find_subnets(self):
        self.subnets = [{"SubnetId": x['SubnetId'],"CidrBlock":x['CidrBlock']}
                    for x in subnets 
                    if x["VpcId"] == self["VpcId"]]

vpc_data = ec2.describe_vpcs()["Vpcs"]
vpcs = [VPC(x) for x in vpc_data]
# for vpc in vpcs:
#     print(vpc['VpcId'])
#     subnets_in_vpc = [', '.join([x['SubnetId'],x['CidrBlock']]) 
#                     for x in subnets 
#                     if x["VpcId"] == vpc["VpcId"]]
#     print(subnets_in_vpc)

# for eni in enis:
#     # check if it is associated with EC2
#     # check if it is associated with ELB
#     # check if it is associated with RDS
#     # VPCE, EFS, TGW, Route53 Resolver,
#     pass

# Going to do it the other way around
# Parse each service, and then mark the ENIs with types when they come up

########################### EC2s
reservations = ec2.describe_instances()['Reservations']
ec2_instances = [y for x in reservations for y in x['Instances'] ]

for ec2_instance in ec2_instances:
    for network_interface in ec2_instance['NetworkInterfaces']:
        enis.attach_resource(eni_id = network_interface["NetworkInterfaceId"], resource_type='EC2', resource_data=network_interface)

########################### ELBs
elb_client = s.client("elb")
elbv2 =  s.client("elbv2")

for elbs in [elb_client.describe_load_balancers()["LoadBalancerDescriptions"], elbv2.describe_load_balancers()["LoadBalancers"]]:
    for elb in elbs:
        attached_enis = [v["NetworkInterfaceId"] for k,v in enis.items() if elb["LoadBalancerName"] in v["Description"]]
        for attached_eni in attached_enis:
            enis.attach_resource(eni_id = attached_eni, resource_type='ELB', resource_data=elb)

vpces = ec2.describe_vpc_endpoints()['VpcEndpoints']
for vpce in vpces:
    attached_enis = [x for x in enis if x in vpce["NetworkInterfaceIds"]]
    # print(attached_enis)
    for attached_eni in attached_enis:
            enis.attach_resource(eni_id = attached_eni, resource_type='VPCE', resource_data=vpce)

# Each transit gateway attachment has an ENI
tgws = ec2.describe_transit_gateway_vpc_attachments()['TransitGatewayVpcAttachments']
for twg in tgws:
    attach_id = twg['TransitGatewayAttachmentId']
    search_string = f'Network Interface for Transit Gateway Attachment {attach_id}'
    attached_enis = [k for k,v in enis.items() if search_string == v['Description']]
    for attached_eni in attached_enis:
            enis.attach_resource(eni_id = attached_eni, resource_type='TWG', resource_data=twg)
# print(tgws)

for ngw in ec2.describe_nat_gateways()['NatGateways']:
    for address in ngw['NatGatewayAddresses']:
        enis.attach_resource(eni_id = address['NetworkInterfaceId'], resource_type='NGW', resource_data={"NAT Gateway":ngw,"IP Addresses":address})

r53 = s.client('route53resolver')
resolvers = r53.list_resolver_endpoints()['ResolverEndpoints']
# print(len(resolvers))
for resolver in resolvers:
    resolver_id = resolver['Id']
    resolver_endpoints = r53.list_resolver_endpoint_ip_addresses(ResolverEndpointId=resolver_id)['IpAddresses']
    for endpoint in resolver_endpoints:
        endpoint_id = endpoint["IpId"]
        search_string = f"Route 53 Resolver: {resolver_id}:{endpoint_id}"
        attached_enis = [k for k,v in enis.items() if search_string == v['Description']]
        # print(f"Found attached enis: {[name for name in attached_enis]}")
        for attached_eni in attached_enis:
            enis.attach_resource(eni_id = attached_eni, resource_type='R53', resource_data={"Resolver Endpoint":resolver,"IP Addresses":endpoint})

efs = s.client("efs",region_name='af-south-1')
file_systems = efs.describe_file_systems()['FileSystems']

efs = s.client("efs",region_name='af-south-1')
file_systems = efs.describe_file_systems()['FileSystems']

for file_system in file_systems:
    file_system_id = file_system['FileSystemId']
    # SecurityAudit doesnt have perms to do efs.describe_access_points() - so I can't get the fsmt value to build the full search_string
    # Can still build the begining of the description - which will match all mount points for this efs
    search_string = f"EFS mount target for {file_system_id}"
    attached_enis = [k for k,v in enis.items() if search_string in v['Description']]
    for attached_eni in attached_enis:
        enis.attach_resource(eni_id = attached_eni, resource_type='EFS', resource_data=file_system)
#efs.describe_access_points()

sgs = ec2.describe_security_groups()['SecurityGroups']

Total subnets: 9

In [3]:
import ipaddress
import itertools
class SUBNET(dict):
    protocol_mapping = {
        '1': 'icmp',
        '6': 'tcp',
        '17': 'udp',
        '58': 'icmpv6'
    }
    # attributes_to_load = "Attachment","SubnetId","Groups"
    def __init__(self, subnet_dict: dict) -> None:
        for k,v in subnet_dict.items():
            # if k in ENI.attributes_to_load:
            self[k] = v
        self.find_enis_in_subnet()
        # print(f"Found {len(self['enis_in_sunet'])} ENIs in {self['SubnetId']}")
        self.get_nacls()
        # self.check_nacls()

    def find_enis_in_subnet(self):
        self['enis_in_sunet'] = [k 
                    for k,x in enis.items() 
                    if x["SubnetId"] == self["SubnetId"]]

    def get_nacls(self):
        self['nacls'] = ec2.describe_network_acls(Filters=[
            {
                'Name': 'association.subnet-id',
                'Values': [
                    self['SubnetId']
                ]
            }
        ])['NetworkAcls']

    def to_ranges(self,iterable):
        iterable = sorted(set(iterable))
        for key, group in itertools.groupby(enumerate(iterable),
                                            lambda t: t[1] - t[0]):
            group = list(group)
            yield group[0][1], group[-1][1]

    def check_nacls(self,ip_address,protocol,from_port,to_port,ingress=True):
        output = {'Allowed':[],'NotAllowed':[]}
        test_port_range = range(int(from_port),int(to_port)+1)
        for nacl in self['nacls']:
            ingress_rules = sorted([entry for entry in nacl['Entries'] if entry['Egress'] == False], key=lambda d: d['RuleNumber'])
            egress_rules = sorted([entry for entry in nacl['Entries'] if entry['Egress'] == True], key=lambda d: d['RuleNumber'])
            if ingress:
                target_rules = ingress_rules
            else:
                target_rules = egress_rules

            for entry in target_rules:
                overlapping_ports = []
                not_overlapping_ports = []
                if ipaddress.ip_address(ip_address) not in ipaddress.IPv4Network(entry['CidrBlock']):
                    continue
                # Protocols for NACLs are always numeric (ie 6 for tcp)
                # Protocols for SGs use common name if it's udp,tcp,icmp,icmpv6 
                # Will convert to SG format
                entry_protocol = SUBNET.protocol_mapping[entry['Protocol']] if entry['Protocol'] in SUBNET.protocol_mapping else entry['Protocol']
                if entry_protocol != '-1' and entry_protocol != protocol:
                    continue
                entry_from_port = entry['PortRange']['From'] if 'PortRange' in entry.keys() else 0
                entry_to_port = entry['PortRange']['To'] if 'PortRange' in entry.keys() else 65535
                entry_port_range = range(entry_from_port,entry_to_port+1)

                overlapping_ports = [x for x in test_port_range if x in entry_port_range]
                test_port_range = [x for x in test_port_range if x not in entry_port_range]

                if not overlapping_ports:
                    continue

                if entry['RuleAction'] == 'allow':
                    output['Allowed'].append({
                        'Ports': list(self.to_ranges(overlapping_ports)),
                        'RuleNumber': entry['RuleNumber']
                    })
                else:
                    output['NotAllowed'].append({
                        'Ports': list(self.to_ranges(overlapping_ports)),
                        'RuleNumber': entry['RuleNumber'],
                        'Default': True if entry == ingress_rules[-1] else False
                    })
        return output

subnet_data = ec2.describe_subnets()["Subnets"]
subnets ={x['SubnetId']: SUBNET(x) for x in subnet_data}

In [4]:
# enis['eni-0062108630e3f8b82'].items()

from lib.graph.nodes import Resource

temp_dict = enis['eni-0062108630e3f8b82']
temp_dict['Name'] = 'eni-0062108630e3f8b82'

test = Resource(labels=['ENI'],properties=temp_dict,key="Name")

test2 = 123


In [13]:
# subnets.keys()
subnets['subnet-05c3ab9fb3a801857'].check_nacls(ip_address='173.245.48.1',protocol='tcp',from_port='0',to_port='65535')

{'Allowed': [{'Ports': [(443, 443)], 'RuleNumber': 110}],
 'NotAllowed': [{'Ports': [(0, 442), (444, 65535)],
   'RuleNumber': 32767,
   'Default': True}]}

In [ ]:
# sgs[0]
import ipaddress

def parse_allowed_enis(allowed_enis,associated_enis):
    for allowed_eni in allowed_enis:
        for associated_eni in associated_enis:
            target_subnet = enis[associated_eni]['SubnetId']
            source_subnet = enis[allowed_eni]['SubnetId']
            if not source_subnet == target_subnet:
                # Need to check the subnet NACL
                target_ip = enis[associated_eni]['PrivateIpAddress']
                source_ip = enis[allowed_eni]['PrivateIpAddress']
                # Check Ingress rules for the target subnet
                target_subnet_ingress_nacl_result = subnets[target_subnet].check_nacls(ip_address=source_ip,protocol=protocol,from_port=from_port,to_port=to_port)
                # Check Egress rules for the source subnet
                source_subnet_egress_nacl_result = subnets[source_subnet].check_nacls(ip_address=target_ip,protocol=protocol,from_port=from_port,to_port=to_port,ingress=False)
                if len(source_subnet_egress_nacl_result['Allowed']) == 0 or len(target_subnet_ingress_nacl_result['Allowed']) == 0:
                    if allowed_eni in allowed_enis:
                        # print(f"Removing {allowed_eni} from the allowed_enis")
                        allowed_enis.remove(allowed_eni)
    return allowed_enis


sg_eni_mapping = {}
# Need to create these neo4j relationships
for sg in sgs:
    sg_eni_mapping[sg['GroupId']] = [k for k,eni in enis.items() if sg['GroupId'] in [attached_sg['GroupId'] for attached_sg in eni['Groups']]]

for sg in sgs[2:3]:
    sg_id = sg['GroupId']
    sg_owner_id = sg['OwnerId']
    for ip_permission in sg['IpPermissions']:
        protocol = ip_permission['IpProtocol']
        if protocol == "-1" or protocol == "50":
            protocol = 'All'
            from_port = 0
            to_port = 65535
        else:
            from_port = ip_permission['FromPort'] if 'FromPort' in ip_permission.keys() else ''
            to_port = ip_permission['ToPort'] if 'ToPort' in ip_permission.keys() else ''
        
        associated_enis = [k for k,eni in enis.items() if sg_id in [attached_sg['GroupId'] for attached_sg in eni['Groups']]]

        ip_ranges = ip_permission['IpRanges']
        for ip_range in ip_ranges:
            cidr_range = ip_range['CidrIp']
            network = ipaddress.IPv4Network(cidr_range) 
            # TODO: Need to loop through each ENI's PrivateIpAddresses instead of just using the primary address
            allowed_enis = [k for k,eni in enis.items() if ipaddress.ip_address(eni['PrivateIpAddress']) in network]
            print(f'{sg_id} allows {len(allowed_enis)} ENIs to access {len(associated_enis)} ENIs ({protocol}: {from_port}-{to_port}) (CIDR Range {cidr_range})')
            allowed_enis = parse_allowed_enis(allowed_enis,associated_enis)
            if len(allowed_enis) != len(after_nacl_allowed_enis):
                print(f'{len(allowed_enis)} ENIs can access {len(associated_enis)} ENIs after parsing NACLs ({protocol}: {from_port}-{to_port}) (SG reference {referenced_sg_id})')

                        # if len(source_subnet_egress_nacl_result['NotAllowed']) or len(target_subnet_ingress_nacl_result['NotAllowed']):
                        #     print(f"Source {allowed_eni} ({source_subnet}) is in a different subnet to Target {associated_eni} ({target_subnet}) and either the ingress or egress NACL rules prevent some of the allowed traffic (according to the SG)")
                        # if len(target_subnet_ingress_nacl_result['NotAllowed']) > 0:
                        #     print(f"Target Subnet Ingress NACL Result: {target_subnet_ingress_nacl_result}")
                        # if len(source_subnet_egress_nacl_result['NotAllowed']) > 0:
                        #     print(f"Source Subnet Egress NACL Result: {source_subnet_egress_nacl_result}")
                

        referenced_sgs = ip_permission['UserIdGroupPairs']
        for referenced_sg in referenced_sgs:
            referenced_sg_id = referenced_sg['GroupId']
            if referenced_sg_id not in sg_eni_mapping.keys():
                continue
            allowed_enis = [k for k,eni in enis.items() if referenced_sg_id in [attached_sg['GroupId'] for attached_sg in eni['Groups']]]
            print(f'{sg_id} allows {len(allowed_enis)} ENIs to access {len(associated_enis)} ENIs ({protocol}: {from_port}-{to_port}) (SG reference {referenced_sg_id})')
            if not referenced_sg['UserId'] == sg_owner_id:
                print(f"External SG referenced in {sg_id}!")
            after_nacl_allowed_enis = parse_allowed_enis(allowed_enis,associated_enis)
            if len(allowed_enis) != len(after_nacl_allowed_enis):
                print(f'{len(allowed_enis)} ENIs can access {len(associated_enis)} ENIs after parsing NACLs ({protocol}: {from_port}-{to_port}) (SG reference {referenced_sg_id})')
            # for allowed_eni in allowed_enis:
            #     for associated_eni in associated_enis:
            #         target_subnet = enis[associated_eni]['SubnetId']
            #         source_subnet = enis[allowed_eni]['SubnetId']
            #         if not source_subnet == target_subnet:
            #             # Need to check the subnet NACL
            #             target_ip = enis[associated_eni]['PrivateIpAddress']
            #             source_ip = enis[allowed_eni]['PrivateIpAddress']
            #             # Check Ingress rules for the target subnet
            #             target_subnet_ingress_nacl_result = subnets[target_subnet].check_nacls(ip_address=source_ip,protocol=protocol,from_port=from_port,to_port=to_port)
            #             # Check Egress rules for the source subnet
            #             source_subnet_egress_nacl_result = subnets[source_subnet].check_nacls(ip_address=target_ip,protocol=protocol,from_port=from_port,to_port=to_port,ingress=False)
            #             if len(source_subnet_egress_nacl_result['Allowed']) == 0 or len(target_subnet_ingress_nacl_result['Allowed']) == 0:
            #                 try:
            #                     allowed_enis.remove(allowed_eni)
            #                 except ValueError:
            #                     # Already removed the ENI when parsing a previously associated ENI
            #                     continue

                        # if len(source_subnet_egress_nacl_result['NotAllowed']) or len(target_subnet_ingress_nacl_result['NotAllowed']):
                        #     print(f"Source {allowed_eni} ({source_subnet}) is in a different subnet to Target {associated_eni} ({target_subnet}) and either the ingress or egress NACL rules prevent some of the allowed traffic (according to the SG)")
                        # if len(target_subnet_ingress_nacl_result['NotAllowed']) > 0:
                        #     print(f"Target Subnet Ingress NACL Result: {target_subnet_ingress_nacl_result}")
                        # if len(source_subnet_egress_nacl_result['NotAllowed']) > 0:
                        #     print(f"Source Subnet Egress NACL Result: {source_subnet_egress_nacl_result}")
                        
            # print(f'{sg_id} allows {allowed_enis} to access {associated_enis} after parsing NACLs ({protocol}: {from_port}-{to_port}) (SG reference {referenced_sg_id})')
                        

In [ ]:
import ipaddress
enis_ip_addresses = [ipaddress.ip_address(eni['PrivateIpAddress']) for k,eni in enis.items()]

In [24]:
enis_ip_addresses[0]

IPv4Address('10.167.2.16')

In [ ]:
for x in [v for k,v in enis.items() if k in vpce["NetworkInterfaceIds"]]:
    print(x)
print(vpce["NetworkInterfaceIds"])
print([x for x in enis])

In [5]:

# print([v['attached_resource_types'] for k,v in enis.items() if 'attached_resource_types' in v.keys()])
# print(f"Completed {len([k for k,v in enis.items() if 'attached_resource_types' in v.keys()])}")
# print(f"Still need to process {len([k for k,v in enis.items() if not 'attached_resource_types' in v.keys()])}")
# [(enis[x]["attached_resource_types"],enis[x]["attached_resources"]) for x in enis if x == 'eni-01f3f6b6b7f159e06'] 
[enis[x] for x in enis if x == 'eni-01f3f6b6b7f159e06'] 

[{'Attachment': {'AttachmentId': 'ela-attach-09546b58ef08d3b57',
   'DeleteOnTermination': False,
   'DeviceIndex': 1,
   'InstanceOwnerId': 'amazon-aws',
   'Status': 'attached'},
  'AvailabilityZone': 'af-south-1a',
  'Description': 'EFS mount target for fs-0da59a09306e7ab0c (fsmt-0efa12722129b5300)',
  'Groups': [{'GroupName': 'default', 'GroupId': 'sg-0cfd8455bd3fb7ba2'},
   {'GroupName': 'terraform-20220712072609866700000001',
    'GroupId': 'sg-01db446416acda057'}],
  'InterfaceType': 'interface',
  'Ipv6Addresses': [],
  'MacAddress': '06:52:a7:c1:9f:62',
  'NetworkInterfaceId': 'eni-01f3f6b6b7f159e06',
  'OwnerId': '981710073011',
  'PrivateDnsName': 'ip-10-167-2-5.af-south-1.compute.internal',
  'PrivateIpAddress': '10.167.2.5',
  'PrivateIpAddresses': [{'Primary': True,
    'PrivateDnsName': 'ip-10-167-2-5.af-south-1.compute.internal',
    'PrivateIpAddress': '10.167.2.5'}],
  'RequesterId': '355971801051',
  'RequesterManaged': True,
  'SourceDestCheck': True,
  'Status': 'i

In [197]:
# [k for k,v in enis.items() if "attached_resource_types" not in v.keys()]
# [(enis[x]["attached_resource_types"],enis[x]["attached_resources"]) for x in enis if x == 'eni-01f3f6b6b7f159e06'] 

output = {}
for x in [enis[x]["attached_resource_types"] for x in enis]:
    for resource_type in x:
        if resource_type not in output.keys():
            output[resource_type] = 1
        else:
            output[resource_type] += 1
print(output)

# print(num_values)

{'EFS': 9, 'VPCE': 24, 'ELB': 12, 'EC2': 14, 'TWG': 3, 'R53': 3}

In [198]:
for subnet in subnets:
    for eni in subnet.enis_in_sunet:
        print(x)

['EC2']

['EC2']

['EC2']

['EC2']

['EC2']

...

['EC2']

['EC2']

['EC2']

['EC2']

['EC2']